<a href="https://colab.research.google.com/github/dookofcrds/90DaysOfCyberSecurity/blob/main/Falcon_7B_GPTQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies and Import Modules

We must build AutoGPTQ from source.

In [1]:
!pip install transformers accelerate einops sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.6 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/PanQiWei/AutoGPTQ
!pip install auto_gptq

Cloning into 'AutoGPTQ'...
remote: Enumerating objects: 3826, done.
remote: Counting objects: 100% (1079/1079), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 3826 (delta 952), reused 948 (delta 908), pack-reused 2747
Receiving objects: 100% (3826/3826), 7.83 MiB | 14.90 MiB/s, done.
Resolving deltas: 100% (2520/2520), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00


In [3]:
import torch
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM

# Load the Model and Tokenizer

This may take a while. It also requires around 3.5GB of RAM and 7.5GB of GPU. It will use 28 GB of disk space.

In [4]:
!pip install auto_gptq --no-cache

In [5]:
model_path = "TheBloke/WizardLM-Uncensored-Falcon-7B-GPTQ"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

In [7]:
model = AutoGPTQForCausalLM.from_quantized(model_path, device="cuda:0", use_triton=False, use_safetensors=True, torch_dtype=torch.float32, trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/TheBloke/WizardLM-Uncensored-Falcon-7B-GPTQ:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.


A new version of the following files was downloaded from https://huggingface.co/TheBloke/WizardLM-Uncensored-Falcon-7B-GPTQ:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


# Your Prompt

You only need to change the prompt below.

In [54]:
prompt = "craft a message to a candidate i think would be perfect for a role make sure to remain concise straight to the point and leverage function over form. im trying to fill start with saying hello and ask them to send their resume and email fi their interested. Keep it short and concise and right to the point add that this job is at a high level tech company that needs a Software Developer whos proficient in Java and HTML in the NY area the salary is 200k and there are wellness benifits make this message unique so it stands out from other recruiters" # Change this to your prompt
prompt_template = f"### Instruction: {prompt}\n### Response:"

This will generate the output:

In [55]:
tokens = tokenizer(prompt_template, return_tensors="pt").input_ids.to("cuda:0")
output = model.generate(input_ids=tokens, max_new_tokens=256, do_sample=False, temperature=0.9, top_k=50, top_p=50)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [56]:
# Print the generated text
print(tokenizer.decode(output[0]))

### Instruction: craft a message to a candidate i think would be perfect for a role make sure to remain concise straight to the point and leverage function over form. im trying to fill start with saying hello and ask them to send their resume and email fi their interested. Keep it short and concise and right to the point add that this job is at a high level tech company that needs a Software Developer whos proficient in Java and HTML in the NY area the salary is 200k and there are wellness benifits make this message unique so it stands out from other recruiters
### Response:Hello! I am reaching out to you as I believe you would be a perfect fit for a Software Developer role at a high-level tech company in the NY area. As per your expertise in Java and HTML, I am confident that you would excel in this role. The salary offered is $200,000, and the company offers wellness benefits as well. If you are interested, please send your resume and email me. Thank you for your time and considerati